In [1]:
%%bash
earthengine authenticate --quiet

Paste the following address into a web browser:

    https://accounts.google.com/o/oauth2/auth?redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&response_type=code&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control

On the web page, please authorize access to your Earth Engine account and copy the authentication code. Next authenticate with the following command:

    earthengine authenticate --authorization-code=PLACE_AUTH_CODE_HERE



In [2]:
%%bash
earthengine authenticate --authorization-code=


Successfully saved authorization token.


In [3]:
import ee

try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

ee.Initialize()

The Earth Engine package initialized successfully!


In [4]:
import numpy as np
import time
import shutil

In [6]:
sensor = 'S2'#L8
res = 'hig'#mid,low

geometry = [[87.802734375,26.745610382199025],
[88.1103515625,21.08450008351735],
[93.1640625,20.3034175184893],
[92.7685546875,26.627818226393046],
[87.802734375,26.745610382199025]]

polygon = ee.Geometry.Polygon(geometry)

if sensor == 'S2':
    collection = ee.ImageCollection('COPERNICUS/S2')
elif sensor == 'L8':
    collection = ee.ImageCollection('LANDSAT/LC8_SR')#//LANDSAT/LC8_L1T_TOA_FMASK')
    
filtered = collection.filterDate('2017-01-01', '2017-03-30').filterBounds(polygon)
perc = collection.reduce(ee.Reducer.percentile([10]))

if res == 'hig':
    result = perc.select('B4_p10', 'B3_p10', 'B2_p10', 'B8_p10')
elif res == 'mid':
    result = perc.select('B5_p10', 'B6_p10', 'B7_p10')

if res == hig:
    scl = 10
elif res == mid:
    scl = 20
    
task = ee.batch.Export.image.toCloudStorage(result, fileNamePrefix='bngldsh_'+res+'_', bucket='testbucket_ron', scale=scl, maxPixels=99524201457, region=geometry)

In [7]:
task.start()